# find and label open ports of a circuit

In [ ]:
import re

dot_source = """
graph new_circuit {
  rankdir=LR;
  node [shape=record];
  C1 [label="{{<o2> o2|<o1> o1} | C1: mzi1 | {<o3> o3|<o4> o4}}"];
  # No edges to connect as there is only one component.
}"""

# dot_source = '''
# graph new_circuit {
#   rankdir=LR;
#   node [shape=record];
#   C1 [label="{{<o2> o2|<o1> o1} | C1: mzi_2x2_heater_tin_cband | {<o3> o3|<o4> o4}}"];
#   C2 [label="{{<o2> o2|<o1> o1} | C2: mzi_2x2_heater_tin_cband | {<o3> o3|<o4> o4}}"];
#   C3 [label="{{<o2> o2|<o1> o1} | C3: mzi_2x2_heater_tin_cband | {<o3> o3|<o4> o4}}"];
#   C4 [label="{{<o2> o2|<o1> o1} | C4: mzi_2x2_heater_tin_cband | {<o3> o3|<o4> o4}}"];
#   C5 [label="{{<o2> o2|<o1> o1} | C5: mzi_2x2_heater_tin_cband | {<o3> o3|<o4> o4}}"];
#   C6 [label="{{<o2> o2|<o1> o1} | C6: mzi_2x2_heater_tin_cband | {<o3> o3|<o4> o4}}"];
#   C7 [label="{{<o2> o2|<o1> o1} | C7: mzi_2x2_heater_tin_cband | {<o3> o3|<o4> o4}}"];
#   C3:o4 -- C7:o1;
#   C3:o3 -- C6:o2;
#   C2:o4 -- C5:o1;
#   C2:o3 -- C4:o2;
#   C1:o4 -- C3:o1;
#   C1:o3 -- C2:o2;
# }
# '''


dot_source = """
graph new_circuit {
  rankdir=LR;
  node [shape=record];
  C1 [label="{{<o2> o2|<o1> o1} | C1: mzi_2x2_heater_tin_cband | {<o3> o3|<o4> o4}}"];
  C2 [label="{{<o2> o2|<o1> o1} | C2: _directional_coupler | {<o3> o3|<o4> o4}}"];
  C1:o4 -- C2:o1;
  C1:o3 -- C2:o2;
}
"""


def find_open_ports(dot_source):
    edges = []
    nodes_ports = {}

    # Parse the graph source to identify edges and node labels
    for line in dot_source.splitlines():
        line = line.strip()
        if "--" in line:
            edge = line.strip(";").split(" -- ")
            edges.append(tuple(edge))
        elif '[label="' in line:
            node = line.split()[0]
            label = re.search(r'\[label="(.*)"\]', line).group(1)
            ports = re.findall(r"<(o\d+)>", label)
            nodes_ports[node] = ports

    # Extract connected ports
    connected_ports = set()
    for edge in edges:
        for endpoint in edge:
            node, port = endpoint.split(":")
            connected_ports.add(f"{node}:{port}")

    # Determine open ports
    open_ports = []
    for node, ports in nodes_ports.items():
        for port in ports:
            if f"{node}:{port}" not in connected_ports:
                open_ports.append(f"{node}:{port}")

    return open_ports


def create_port_dict(open_ports):
    port_dict = {}
    for i, open_port in enumerate(open_ports):
        port_dict[f"o{i+1}"] = open_port.replace(":", ",")
    return port_dict


open_ports = find_open_ports(dot_source)
port_dict = create_port_dict(open_ports)

print(open_ports)
print(port_dict)

# clean up and port ordering

In [ ]:
dot_graph = """
graph new_circuit {
  rankdir=LR;
  node [shape=record];
  C1 [label="{{<o2> o2|<o1> o1} | C1: mzi_2x2_heater_tin_cband | {<o3> o3|<o4> o4}}"];
  C2 [label="{{<o2> o2|<o1> o1} | C2: _directional_coupler | {<o3> o3|<o4> o4}}"];
  C1:o4 -- C2:o1;
  C1:o3 -- C2:o2;
}
"""


def extract_ports(dot_graph):
    pattern = re.compile(
        r'(\w+)\s*\[label\s*=\s*"\{\{([^}]+)\}\s*\|\s*([^|]+)\s*\|\s*\{([^}]+)\}\}"\];'
    )
    ports = []

    for match in re.findall(pattern, dot_graph):
        node = match[0]
        port_groups = match[1], match[3]

        for group in port_groups:
            for port in re.findall(r"<(\w+)>", group):
                ports.append(f"{node}:{port}")

    return ports


ports = extract_ports(dot_graph)
print(ports)